<a href="https://colab.research.google.com/github/Homaoa/NLP-for-a-Hotel-Reviews-Data-Set/blob/main/NLP_model_for_hotel_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
url = '/tripadvisor_hotel_reviews.csv'
df = pd.read_csv(url)
df.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20491 entries, 0 to 20490
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  20491 non-null  object
 1   Rating  20491 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 320.3+ KB


In [ ]:
#There are 20491 reviews in this dataset

In [4]:
df['Rating'].value_counts()

5    9054
4    6039
3    2184
2    1793
1    1421
Name: Rating, dtype: int64

In [ ]:
#Ratings are between 1 to 5

In [ ]:
#Now, I have to clean and prepare the data for the bag of words model
#I replce anything other than letters with space
#I make all the letters lower
#Then I apply stemming to get the root of the words
#I also take negative words out of the stopwords, because I want the model to consider it. It can affect the reviews

In [11]:
negative_words = ["not", "isn't", "no", "don’t", "should", "should’ve", "ain", "aren", "aren’t",
                       "couldn", "couldn’t", "didn", "didn’t", "doesn", "doesn’t", "hadn", "hadn’t",
                       "hasn", "hasn’t", "haven", "haven’t", "isn", "isn’t", "mightn", "mightn’t",
                       "mustn", "mustn’t", "needn", "needn’t", "shan", "shan’t", "shouldn", "shouldn’t",
                       "wasn", "wasn’t", "weren", "weren’t", "won", "won’t", "wouldn", "wouldn’t"]

In [12]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
all_stopwords = stopwords.words('english')
all_stopwords_updated = [word for word in all_stopwords if not word in negative_words]

corpus = []
for i in range(0,20491):
  review = re.sub('[^a-zA-Z]',' ',df['Review'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set (all_stopwords_updated)]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Now I develop the bag of words model

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=20000)
X = cv.fit_transform(corpus).toarray()
y = df['Rating']

In [ ]:
#I can tell CountVectorizer that how many words it should consider. First I will check how many words in total I have by using len function

In [14]:
len(X[0])

20000

In [ ]:
#It is 35692 words. I can reduce that, because many of those words can be names, and things like that that are not useful for the model,
#they just make the matrix of features bigger. So I set max_features eaqual to 20000

In [ ]:
#Now I split the data intop train and test to apply classification models

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
#Now I train the naive bayes model on the training set and get prediction

In [16]:
from sklearn.naive_bayes import GaussianNB
classifier=GaussianNB()
classifier.fit(X_train,y_train)

GaussianNB()

In [17]:
y_pred=classifier.predict(X_test)

In [18]:
from sklearn.metrics import confusion_matrix , accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[ 88  54  20  71  52]
 [ 73  53  34 125  70]
 [114  71  71 150  65]
 [192 163 183 393 272]
 [300 207 234 542 502]]


0.270065869724323

In [ ]:
#Now I try logistic regression

In [19]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [20]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix , accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[ 176   81   14   10    4]
 [  73  157   81   34   10]
 [  17  108  130  174   42]
 [   6   37  137  566  457]
 [   2   15   23  387 1358]]


0.582337155403757

In [ ]:
#Now I try knn model

In [21]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [25]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix , accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[ 112   33    7   46   87]
 [  71   51   31  102  100]
 [  42   31   52  204  142]
 [  35   22   75  547  524]
 [  25   23   32  523 1182]]


0.4742620151256404

In [ ]:
#I also want to try the random forest model too

In [26]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [27]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix , accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 122   53   11   38   61]
 [  56   45   32   98  124]
 [  25   38   26  195  187]
 [   9   13   55  459  667]
 [  12   11   24  395 1343]]


0.48670407416443034